<a href="https://colab.research.google.com/github/prtk0007/Chest_Xray_detection/blob/exp_binary/project_ray_5_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [2]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:30<00:00, 26.5MB/s]
100% 2.29G/2.29G [00:30<00:00, 80.3MB/s]


In [ ]:
!unzip \*.zip

In [5]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [6]:
IMAGE_SIZE = [224, 224]

train_path = '/content/chest_xray/chest_xray/train'
valid_path = '/content/chest_xray/chest_xray/test'

In [7]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80142336/80134624 [==============================] - 1s 0us/step


In [8]:
for layer in vgg.layers:
    layer.trainable = False

In [9]:
folders = glob('/content/chest_xray/chest_xray/train/*')
folders

['/content/chest_xray/chest_xray/train/NORMAL',
 '/content/chest_xray/chest_xray/train/PNEUMONIA']

In [ ]:
vgg.summary()

In [11]:
from keras.layers import Dropout
x = Flatten(name='flatten')(vgg.output)
x = Dense(256, activation='relu', name='fc1')(x)
x = Dropout(0.5, name='drop1')(x)
x = Dense(256, activation='relu', name='fc2')(x)
x = Dropout(0.5, name='drop2')(x)
x = Dense(1, activation='sigmoid', name='predictions')(x)

In [12]:
model = Model(inputs=vgg.input, outputs=x)

In [ ]:
model.summary()

In [14]:
from keras.optimizers import Adam

model.compile(
  loss='binary_crossentropy',
  optimizer= 'adam',
  metrics=['accuracy']
)

In [15]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255) 
val_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
training_set = train_datagen.flow_from_directory('/content/chest_xray/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 5216 images belonging to 2 classes.


In [17]:
test_set = test_datagen.flow_from_directory('/content/chest_xray/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode = 'binary')

Found 624 images belonging to 2 classes.


In [18]:
val_set = val_datagen.flow_from_directory('/content/chest_xray/chest_xray/val',
                                            target_size = (224, 224),
                                            class_mode = 'binary')

Found 16 images belonging to 2 classes.


In [27]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor='val_accuracy', mode='min', patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 7, verbose=1,factor=0.3, min_lr=0.001)
callbacks_list = [ early, learning_rate_reduction]

In [28]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  callbacks=callbacks_list
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
163/163 [==============================] - 104s 637ms/step - loss: 0.3068 - accuracy: 0.7743 - val_loss: 0.4461 - val_accuracy: 0.8782
Epoch 2/20
163/163 [==============================] - 103s 631ms/step - loss: 0.3062 - accuracy: 0.8503 - val_loss: 0.4258 - val_accuracy: 0.8846
Epoch 3/20
163/163 [==============================] - 103s 633ms/step - loss: 0.2999 - accuracy: 0.8610 - val_loss: 0.4448 - val_accuracy: 0.8734
Epoch 4/20
163/163 [==============================] - 103s 630ms/step - loss: 0.2937 - accuracy: 0.8608 - val_loss: 0.4305 - val_accuracy: 0.8782
Epoch 5/20
163/163 [==============================] - 103s 629ms/step - loss: 0.2947 - accuracy: 0.8652 - val_loss: 0.4387 - val_accuracy: 0.8494
Epoch 6/20
163/163 [==============================] - 103s 632ms/step - loss: 0.2928 - accuracy: 0.8656 - val_loss: 0.4039 - val_accuracy: 0.8942
Epoch 7/20
163/163 [==============================] - 103s 630ms/step - loss: 0.2926 - accuracy: 0.8616 - val_loss: 0.4101 -

In [29]:
model.evaluate(training_set)

163/163 [==============================] - 97s 595ms/step - loss: 0.2069 - accuracy: 0.9574


[0.20689912140369415, 0.9574386477470398]

In [30]:
model.evaluate(test_set)

20/20 [==============================] - 6s 268ms/step - loss: 0.3827 - accuracy: 0.8942


[0.38271185755729675, 0.8942307829856873]

In [23]:
model.evaluate(val_set)

1/1 [==============================] - 0s 212ms/step - loss: 0.5598 - accuracy: 0.5000


[0.5598418712615967, 0.5]

In [ ]:
 pred = model.predict_generator(val_set)


In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Kaggle/model_vgg19.h5')

In [ ]:
import pandas as pd 
import cv2                 
import numpy as np         
import os                  
from random import shuffle
from tqdm import tqdm  
import scipy
import skimage
from skimage.transform import resize

In [ ]:
TRAIN_DIR = "/content/chest_xray/chest_xray/train/"
TEST_DIR =  "/content/chest_xray/chest_xray/test/"

In [ ]:
def get_label(Dir):
    for nextdir in os.listdir(Dir):
        if not nextdir.startswith('.'):
            if nextdir in ['NORMAL']:
                label = 0
            elif nextdir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
    return nextdir, label

In [ ]:
def preprocessing_data(Dir):
    X = []
    y = []
    
    for nextdir in os.listdir(Dir):
        nextdir, label = get_label(Dir)
        temp = Dir + nextdir
        
        for image_filename in tqdm(os.listdir(temp)):
            path = os.path.join(temp + '/' , image_filename)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = skimage.transform.resize(img, (224, 224, 3))
                img = np.asarray(img)
                X.append(img)
                y.append(label)
            
    X = np.asarray(X)
    y = np.asarray(y)
    
    return X,y

In [ ]:
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['NORMAL']:
                label = 0
            elif nextDir in ['PNEUMONIA']:
                label = 1
            else:
                label = 2
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                img = cv2.imread(temp + '/' + file)
                if img is not None:
                    img = skimage.transform.resize(img, (224, 224, 3))
                    #img_file = scipy.misc.imresize(arr=img_file, size=(150, 150, 3))
                    img = np.asarray(img)
                    X.append(img)
                    y.append(label)
                    
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [ ]:
X_test , y_test = get_data(TEST_DIR)

In [ ]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(y_test, 2)


In [ ]:
print(y_test.shape)

(624, 2)


In [ ]:
from sklearn.metrics import confusion_matrix
pred = model.predict(X_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

In [ ]:
cm = confusion_matrix(y_true, pred)
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(5, 5))
plt.show()

In [ ]:
print('True Positive  (TP) = ', cm[0][0])
print('False Positive (FP) = ', cm[0][1])
print('False Negative (FN) = ', cm[1][0])
print('True Negative  (TN) = ', cm[1][1])
print('-'*26)

In [ ]:
if (cm[0][0] + cm[0][1])!=0 and (cm[0][0], cm[1][0])!=0:
      accuracy=  round((cm[1][1] + cm[0][0]) / (cm[0][0] + cm[1][0] + cm[0][1] + cm[1][1])*100, 2)
      precision= round((cm[1][1] / (cm[1][1] + cm[0][1]))*100, 2) 
      recall   = round((cm[1][1] / (cm[1][1] + cm[1][0]))*100, 2) 
      f1_score = round(2*((precision*recall)/(precision+recall)), 2)  
      print(f'accuracy Score: {accuracy}%')     
      print(f'Precision Score: {precision}%')
      print(f'Recall Score: {recall}%')
      print(f'F1 Score: {f1_score}%')
      print('='*50) 
      print('\n')